In [67]:
%run /Users/tyamgin/Projects/mlbootcamp/championship19/cv.ipynb
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/data'
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/res'
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 250)

In [4]:
train_data = pd.read_pickle(output_path + '/train_data')
test_data = pd.read_pickle(output_path + '/test_data')

In [41]:
owner_stat = pd.concat((train_data, test_data), sort=False)[['metadata_ownerId', 'objectId']] \
    .groupby('metadata_ownerId').agg({'objectId': 'count'})
owner_stat.rename(columns={'objectId': 'owner_count'}, inplace=True)

In [48]:
author_stat = pd.concat((train_data, test_data), sort=False)[['metadata_authorId', 'objectId']] \
    .groupby('metadata_authorId').agg({'objectId': 'count'})
author_stat.rename(columns={'objectId': 'author_count'}, inplace=True)

In [42]:
train_data = train_data.join(owner_stat, how='left', on='metadata_ownerId')
test_data = test_data.join(owner_stat, how='left', on='metadata_ownerId')

In [49]:
train_data = train_data.join(author_stat, how='left', on='metadata_authorId')
test_data = test_data.join(author_stat, how='left', on='metadata_authorId')

In [53]:
def create_features(data):
    data = data.copy()
    
    data.drop(['audit_experiment', 'date'], 1, inplace=True)
    
    if 'liked' in data.columns:
        data['label'] = data['liked']
        data.drop(['liked', 'clicked', 'viewed', 'disliked', 'reshared', 'ignored', 'commented', 'complaint', 
                   'unliked'], 1, inplace=True)
        
    '''
    NA's mean:
    membership_joinRequestDate 0.9844882926409128
    auditweights_closed 0.9999999453150741
    auditweights_feedOwner_RECOMMENDED_GROUP 0.9876739630029133
    auditweights_friendCommentFeeds 0.9937154989384289
    auditweights_friendCommenters 0.9937154989384289
    auditweights_friendLikes 0.7103393609792976
    auditweights_friendLikes_actors 0.7103393609792976
    auditweights_hasDetectedText 0.9999950783566632
    auditweights_hasText 0.9900832714710108
    auditweights_likersFeedStats_hyper 0.7103393609792976
    auditweights_likersSvd_prelaunch_hyper 0.8568170912267606
    auditweights_notOriginalPhoto 0.9537070227749045
    auditweights_onlineVideo 0.9986092529628976
    auditweights_processedVideo 0.9569228245311109
    auditweights_source_MOVIE_TOP 0.9999782353994665
    auditweights_userOwner_CREATE_COMMENT 0.9674945691032902
    auditweights_userOwner_CREATE_IMAGE 0.9992598395270847
    auditweights_userOwner_CREATE_LIKE 0.6409860785849728
    auditweights_userOwner_IMAGE 0.8148977596952955
    auditweights_userOwner_TEXT 0.6919283682154804
    auditweights_userOwner_UNKNOWN 0.991493759766386
    auditweights_userOwner_USER_FEED_REMOVE 0.9767927564347069
    auditweights_userOwner_VIDEO 0.9964757205764339
    auditweights_x_ActorsRelations 0.9143301028213321
    auditweights_likersSvd_spark_hyper 0.8535222697525371
    '''
    
    if 'auditweights_closed' in data.columns:
        data.drop('auditweights_closed', 1, inplace=True)
    
    data.drop(['membership_joinRequestDate', 'auditweights_feedOwner_RECOMMENDED_GROUP', 
               'auditweights_friendCommentFeeds', 'auditweights_friendCommenters', 'auditweights_hasDetectedText',
               'auditweights_hasText', 'auditweights_onlineVideo', 'auditweights_source_MOVIE_TOP',
               'auditweights_userOwner_CREATE_IMAGE', 'auditweights_userOwner_UNKNOWN',
               'auditweights_userOwner_VIDEO'], 1, inplace=True)
    
    data['old'] = data['audit_timestamp'] - data['metadata_createdAt']
    data.drop(['audit_timestamp', 'metadata_createdAt'], 1, inplace=True)
    
    #'metadata_ownerId', 'metadata_authorId'

    return data

In [6]:
train = create_features(train_data)

In [7]:
class LgbModel(MyModel):
    def fit(self, data):
        lgb_train = lgb.Dataset(self.get_X(data), data['label'].values)
        
        #if self.verbose >= 2:
        print('Starting train: %s' % datetime.datetime.now())
        params = self.params.copy()
        num_boost_round = params['num_boost_round']
        del params['num_boost_round']
        params['metric'] = 'auc'
        self.model = lgb.train(
            params,
            lgb_train,
            num_boost_round=num_boost_round
        )
    def predict(self, X):
        proba = self.model.predict(self.get_X(X))
        return proba


In [43]:
train

,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timePassed,audit_resourceType,metadata_ownerId,metadata_ownerType,metadata_authorId,metadata_applicationId,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,metadata_numVideos,metadata_platform,metadata_totalVideoLength,userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_CREATE,userOwnerCounters_PHOTO_VIEW,userOwnerCounters_PHOTO_PIN_BATCH_CREATE,userOwnerCounters_PHOTO_PIN_UPDATE,userOwnerCounters_USER_PRESENT_SEND,userOwnerCounters_UNKNOWN,userOwnerCounters_CREATE_TOPIC,userOwnerCounters_CREATE_IMAGE,userOwnerCounters_CREATE_MOVIE,userOwnerCounters_CREATE_COMMENT,userOwnerCounters_CREATE_LIKE,userOwnerCounters_TEXT,userOwnerCounters_IMAGE,userOwnerCounters_VIDEO,membership_status,membership_statusUpdateDate,membership_joinDate,user_create_date,user_birth_date,user_gender,user_status,user_ID_country,user_ID_Location,user_is_active,user_is_deleted,user_is_abused,user_is_activated,user_change_datime,user_is_semiactivated,user_region,objectId,auditweights_ageMs,auditweights_ctr_gender,auditweights_ctr_high,auditweights_ctr_negative,auditweights_dailyRecency,auditweights_feedStats,auditweights_friendLikes,auditweights_friendLikes_actors,auditweights_isRandom,auditweights_likersFeedStats_hyper,auditweights_likersSvd_prelaunch_hyper,auditweights_matrix,auditweights_notOriginalPhoto,auditweights_numDislikes,auditweights_numLikes,auditweights_numShows,auditweights_processedVideo,auditweights_svd_prelaunch,auditweights_svd_spark,auditweights_userAge,auditweights_userOwner_CREATE_COMMENT,auditweights_userOwner_CREATE_LIKE,auditweights_userOwner_IMAGE,auditweights_userOwner_TEXT,auditweights_userOwner_USER_FEED_REMOVE,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,has_cheated_text,has_companions,has_detected_text,has_music,has_photos,has_pins,has_polls,has_text,has_urls,has_videos,is_censored,is_disgusting,is_external_share,is_gif,is_internal_group_share,is_internal_share,is_part_of_album,is_part_of_topic,is_promo,label,old
0,138,0,16788021,1,1,3651351,8,2154,0,876165,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.826869,0.000000,0.000000,0.000000,1.0,1.234719e+12,1.234719e+12,1.182856e+12,1164.0,2.0,128.0,1.041453e+10,3897824.0,1.0,0.0,0.0,1.0,1.445518e+12,0.0,3.0,16788021,35742344.0,0.009836,0.065857,0.001093,0.973046,0.1,NaN,NaN,1.0,NaN,NaN,0.021505,NaN,1.0,8.0,815.0,NaN,0.603160,NaN,44.0,NaN,0.689975,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,35742619
1,222,0,23852723,33,0,3937907,8,23080,0,755833,0,0,8,0,44,6,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.545485,0.545485,0.000000,0.000000,1.0,1.492272e+12,1.492272e+12,1.182870e+12,-8289.0,2.0,192.0,1.042408e+10,2900163.0,1.0,0.0,0.0,1.0,1.445518e+12,0.0,246.0,23852723,130239136.0,0.023984,0.135539,0.004631,0.923622,0.1,NaN,NaN,1.0,NaN,NaN,0.024493,NaN,1036.0,6064.0,223594.0,NaN,0.675362,NaN,70.0,NaN,0.622459,NaN,0.622459,NaN,NaN,NaN,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,130239978
2,384,0,12454221,15,0,1517508395567,8,39056,0,264946,0,0,1,0,138,18,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,2.402731,0.960400,0.000000,0.000000,1.0,1.398106e+12,1.398106e+12,1.174492e+12,-22615.0,2.0,192.0,1.040617e+10,2244

In [68]:
%%time
#################################### PARTIAL TRAIN
for min_data_in_leaf in (20,):
    for lambda_l2 in (0.0,):
        for learning_rate in (0.15,):
            for feature_fraction in (1,):
                for num_boost_round in (1000,):
                    for num_leaves in (20,):
                        print((min_data_in_leaf,lambda_l2,learning_rate,feature_fraction,num_boost_round,num_leaves))
                        cross_validation(LgbModel({
                            'objective': 'binary',
                            'boosting_type': 'gbdt',
                            'min_data_in_leaf': min_data_in_leaf,
                            'lambda_l2': lambda_l2,
                            'num_leaves': num_leaves,
                            'learning_rate': learning_rate,
                            'feature_fraction': feature_fraction,
                            'bagging_fraction': 1,
                            'bagging_freq': 5,
                            'num_boost_round': num_boost_round,
                            'verbose': 0
                        }), train[train.instanceId_userId % 15 == 6], n_iters=1, verbose=2)

(20, 0.0, 0.15, 1, 1000, 30)
KFold(n_splits=5, random_state=2707, shuffle=True)
Prepare data: 2019-03-05 00:35:39.082312
Fit: 2019-03-05 00:35:46.665156
Starting train: 2019-03-05 00:35:53.220745
Predict: 2019-03-05 00:40:06.685902
Auc: 2019-03-05 00:40:19.952752
 0 - 1 : 0.7172, mean=0.7172
Prepare data: 2019-03-05 00:40:20.175968
Fit: 2019-03-05 00:40:30.206341
Starting train: 2019-03-05 00:40:36.792624
Predict: 2019-03-05 00:45:09.124589
Auc: 2019-03-05 00:45:20.521009
 0 - 2 : 0.7186, mean=0.7179
Prepare data: 2019-03-05 00:45:20.781376
Fit: 2019-03-05 00:45:28.927086
Starting train: 2019-03-05 00:45:33.895751


KeyboardInterrupt: 

In [51]:
%%time
#################################### PARTIAL TRAIN
for min_data_in_leaf in (20,):
    for lambda_l2 in (0.0,):
        for learning_rate in (0.15,):
            for feature_fraction in (1,):
                for num_boost_round in (1000,):
                    for num_leaves in (20,):
                        print((min_data_in_leaf,lambda_l2,learning_rate,feature_fraction,num_boost_round,num_leaves))
                        cross_validation(LgbModel({
                            'objective': 'binary',
                            'boosting_type': 'gbdt',
                            'min_data_in_leaf': min_data_in_leaf,
                            'lambda_l2': lambda_l2,
                            'num_leaves': num_leaves,
                            'learning_rate': learning_rate,
                            'feature_fraction': feature_fraction,
                            'bagging_fraction': 1,
                            'bagging_freq': 5,
                            'num_boost_round': num_boost_round,
                            'verbose': 0
                        }), train[train.instanceId_userId % 15 == 6], n_iters=1, verbose=2)

(20, 0.0, 0.15, 1, 1000, 20)
KFold(n_splits=5, random_state=2707, shuffle=True)
Prepare data: 2019-03-04 23:20:11.852720
Fit: 2019-03-04 23:20:21.804574
Starting train: 2019-03-04 23:20:32.241045
Predict: 2019-03-04 23:24:36.973813
Auc: 2019-03-04 23:24:48.211610
 0 - 1 : 0.7164, mean=0.7164
Prepare data: 2019-03-04 23:24:48.430732
Fit: 2019-03-04 23:24:56.696690
Starting train: 2019-03-04 23:25:01.090636
Predict: 2019-03-04 23:29:14.890042
Auc: 2019-03-04 23:29:26.785149
 0 - 2 : 0.7198, mean=0.7181
Prepare data: 2019-03-04 23:29:27.030821


KeyboardInterrupt: 

In [60]:
train.columns.tolist()

['instanceId_userId',
 'instanceId_objectType',
 'instanceId_objectId',
 'audit_pos',
 'audit_clientType',
 'audit_timePassed',
 'audit_resourceType',
 'metadata_ownerId',
 'metadata_ownerType',
 'metadata_authorId',
 'metadata_applicationId',
 'metadata_numCompanions',
 'metadata_numPhotos',
 'metadata_numPolls',
 'metadata_numSymbols',
 'metadata_numTokens',
 'metadata_numVideos',
 'metadata_platform',
 'metadata_totalVideoLength',
 'userOwnerCounters_USER_FEED_REMOVE',
 'userOwnerCounters_USER_PROFILE_VIEW',
 'userOwnerCounters_VOTE_POLL',
 'userOwnerCounters_USER_SEND_MESSAGE',
 'userOwnerCounters_USER_DELETE_MESSAGE',
 'userOwnerCounters_USER_INTERNAL_LIKE',
 'userOwnerCounters_USER_INTERNAL_UNLIKE',
 'userOwnerCounters_USER_STATUS_COMMENT_CREATE',
 'userOwnerCounters_PHOTO_COMMENT_CREATE',
 'userOwnerCounters_MOVIE_COMMENT_CREATE',
 'userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE',
 'userOwnerCounters_COMMENT_INTERNAL_LIKE',
 'userOwnerCounters_USER_FORUM_MESSAGE_CREATE',
 'us

In [69]:
small_subsample = train[train.instanceId_userId % 21 == 6]
print(small_subsample.shape)

(2604444, 113)


In [70]:
feature_selection(small_subsample, LgbModel({
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 20,
    'lambda_l2': 0.0,
    'num_leaves': 20,
    'learning_rate': 0.15,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 500,
    'verbose': 0
}), [
 'userOwnerCounters_USER_FEED_REMOVE',
 'userOwnerCounters_USER_PROFILE_VIEW',
 'userOwnerCounters_VOTE_POLL',
 'userOwnerCounters_USER_SEND_MESSAGE',
 'userOwnerCounters_USER_DELETE_MESSAGE',
 'userOwnerCounters_USER_INTERNAL_LIKE',
 'userOwnerCounters_USER_INTERNAL_UNLIKE',
 'userOwnerCounters_USER_STATUS_COMMENT_CREATE',
 'userOwnerCounters_PHOTO_COMMENT_CREATE',
 'userOwnerCounters_MOVIE_COMMENT_CREATE',
 'userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE',
 'userOwnerCounters_COMMENT_INTERNAL_LIKE',
 'userOwnerCounters_USER_FORUM_MESSAGE_CREATE',
 'userOwnerCounters_PHOTO_MARK_CREATE',
 'userOwnerCounters_PHOTO_VIEW',
 'userOwnerCounters_PHOTO_PIN_BATCH_CREATE',
 'userOwnerCounters_PHOTO_PIN_UPDATE',
 'userOwnerCounters_USER_PRESENT_SEND',
 'userOwnerCounters_UNKNOWN',
 'userOwnerCounters_CREATE_TOPIC',
 'userOwnerCounters_CREATE_IMAGE',
 'userOwnerCounters_CREATE_MOVIE',
 'userOwnerCounters_CREATE_COMMENT',
 'userOwnerCounters_CREATE_LIKE',
 'userOwnerCounters_TEXT',
 'userOwnerCounters_IMAGE',
 'userOwnerCounters_VIDEO',
 'membership_status',
 'membership_statusUpdateDate',
 'membership_joinDate',
 'user_create_date',
 'user_birth_date',
 'user_gender',
 'user_status',
 'user_ID_country',
 'user_ID_Location',
 'user_is_active',
 'user_is_deleted',
 'user_is_abused',
 'user_is_activated',
 'user_change_datime',
 'user_is_semiactivated',
 'user_region',
 'auditweights_ageMs',
 'auditweights_ctr_gender',
 'auditweights_ctr_high',
 'auditweights_ctr_negative',
 'auditweights_dailyRecency',
 'auditweights_feedStats',
 'auditweights_friendLikes',
 'auditweights_friendLikes_actors',
 'auditweights_isRandom',
 'auditweights_likersFeedStats_hyper',
 'auditweights_likersSvd_prelaunch_hyper',
 'auditweights_matrix',
 'auditweights_notOriginalPhoto',
 'auditweights_numDislikes',
 'auditweights_numLikes',
 'auditweights_numShows',
 'auditweights_processedVideo',
 'auditweights_svd_prelaunch',
 'auditweights_svd_spark',
 'auditweights_userAge',
 'auditweights_userOwner_CREATE_COMMENT',
 'auditweights_userOwner_CREATE_LIKE',
 'auditweights_userOwner_IMAGE',
 'auditweights_userOwner_TEXT',
 'auditweights_userOwner_USER_FEED_REMOVE',
 'auditweights_x_ActorsRelations',
 'auditweights_likersSvd_spark_hyper',
 'has_cheated_text',
 'has_companions',
 'has_detected_text',
 'has_music',
 'has_photos',
 'has_pins',
 'has_polls',
 'has_text',
 'has_urls',
 'has_videos',
 'is_censored',
 'is_disgusting',
 'is_external_share',
 'is_gif',
 'is_internal_group_share',
 'is_internal_share',
 'is_part_of_album',
 'is_part_of_topic',
 'is_promo',
 'old',
 'owner_count',
 'author_count',
 'instanceId_objectType',
 'audit_pos',
 'audit_clientType',
 'audit_timePassed',
 'audit_resourceType',
 'metadata_ownerType',
 'metadata_applicationId',
 'metadata_numCompanions',
 'metadata_numPhotos',
 'metadata_numPolls',
 'metadata_numSymbols',
 'metadata_numTokens',
 'metadata_numVideos',
 'metadata_platform',
 'metadata_totalVideoLength'
], ['instanceId_userId', 'label', 'objectId'])

KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 00:47:07.160346
 0 - 1 : 0.7166, mean=0.7166
Starting train: 2019-03-05 00:48:54.107308
 0 - 2 : 0.7144, mean=0.7155
Starting train: 2019-03-05 00:50:56.223093
 0 - 3 : 0.7169, mean=0.7159
Starting train: 2019-03-05 00:52:51.101816
 0 - 4 : 0.7181, mean=0.7165
Starting train: 2019-03-05 00:54:46.012698
 0 - 5 : 0.7148, mean=0.7162
Start with score=0.716163: userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_CREATE,userOwnerCounters_PHO

 0 - 1 : 0.7166, mean=0.7166
Starting train: 2019-03-05 02:31:39.333346
 0 - 2 : 0.7144, mean=0.7155
Starting train: 2019-03-05 02:33:17.928780
 0 - 3 : 0.7169, mean=0.7159
Starting train: 2019-03-05 02:34:52.501164
 0 - 4 : 0.7181, mean=0.7165
Starting train: 2019-03-05 02:36:29.342517
 0 - 5 : 0.7148, mean=0.7162
Try to remove userOwnerCounters_USER_FORUM_MESSAGE_CREATE
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 02:38:15.267907
 0 - 1 : 0.7166, mean=0.7166
Starting train: 2019-03-05 02:39:50.903004
 0 - 2 : 0.7144, mean=0.7155
Starting train: 2019-03-05 02:41:30.733420
 0 - 3 : 0.7169, mean=0.7159
Starting train: 2019-03-05 02:43:06.705365
 0 - 4 : 0.7181, mean=0.7165
Starting train: 2019-03-05 02:44:43.720199
 0 - 5 : 0.7148, mean=0.7162
Try to remove userOwnerCounters_PHOTO_MARK_CREATE
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 02:46:29.990042
 0 - 1 : 0.7166, mean=0.7166
Starting train: 2019-03-05 02:48:07.023140


KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 03:51:24.395850
 0 - 1 : 0.7164, mean=0.7164
Starting train: 2019-03-05 03:52:59.616712
 0 - 2 : 0.7137, mean=0.7151
Starting train: 2019-03-05 03:54:31.110411
 0 - 3 : 0.7180, mean=0.7161
Starting train: 2019-03-05 03:56:06.013711
 0 - 4 : 0.7183, mean=0.7166
Starting train: 2019-03-05 03:57:39.639642
 0 - 5 : 0.7159, mean=0.7165
Found [userOwnerCounters_CREATE_MOVIE] score=0.716467: userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_

 0 - 5 : 0.7154, mean=0.7164
Try to remove user_ID_country
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 05:33:45.158098
 0 - 1 : 0.7172, mean=0.7172
Starting train: 2019-03-05 05:35:16.049723
 0 - 2 : 0.7143, mean=0.7158
Starting train: 2019-03-05 05:36:46.433498
 0 - 3 : 0.7169, mean=0.7161
Starting train: 2019-03-05 05:38:26.399211
 0 - 4 : 0.7177, mean=0.7165
Starting train: 2019-03-05 05:40:04.491148
 0 - 5 : 0.7151, mean=0.7162
Try to remove user_ID_Location
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 05:41:45.334035
 0 - 1 : 0.7164, mean=0.7164
Starting train: 2019-03-05 05:43:17.644291
 0 - 2 : 0.7137, mean=0.7150
Starting train: 2019-03-05 05:44:53.694475
 0 - 3 : 0.7174, mean=0.7158
Starting train: 2019-03-05 05:46:33.946785
 0 - 4 : 0.7181, mean=0.7164
Starting train: 2019-03-05 05:48:06.186318
 0 - 5 : 0.7161, mean=0.7163
Try to remove user_is_active
KFold(n_splits=5, random_state=430, shuffle=True)
Starting tr

Starting train: 2019-03-05 07:57:07.549661
 0 - 2 : 0.7131, mean=0.7146
Starting train: 2019-03-05 07:58:38.637254
 0 - 3 : 0.7168, mean=0.7153
Starting train: 2019-03-05 08:00:10.362701
 0 - 4 : 0.7190, mean=0.7163
Starting train: 2019-03-05 08:01:43.869812
 0 - 5 : 0.7156, mean=0.7161
Try to remove auditweights_likersSvd_prelaunch_hyper
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 08:03:26.757030
 0 - 1 : 0.7167, mean=0.7167
Starting train: 2019-03-05 08:04:58.835235
 0 - 2 : 0.7135, mean=0.7151
Starting train: 2019-03-05 08:06:29.890729
 0 - 3 : 0.7169, mean=0.7157
Starting train: 2019-03-05 08:07:56.350433
 0 - 4 : 0.7183, mean=0.7164
Starting train: 2019-03-05 08:09:33.465739
 0 - 5 : 0.7156, mean=0.7162
Try to remove auditweights_matrix
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 08:11:09.595451
 0 - 1 : 0.7155, mean=0.7155
Starting train: 2019-03-05 08:12:41.995956
 0 - 2 : 0.7127, mean=0.7141
Starting train: 2019-

KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 10:41:49.081510
 0 - 1 : 0.7168, mean=0.7168
Starting train: 2019-03-05 10:43:34.556279
 0 - 2 : 0.7137, mean=0.7152
Starting train: 2019-03-05 10:45:20.609317
 0 - 3 : 0.7172, mean=0.7159
Starting train: 2019-03-05 10:47:05.259450
 0 - 4 : 0.7181, mean=0.7164
Starting train: 2019-03-05 10:48:54.012628
 0 - 5 : 0.7160, mean=0.7164
Try to remove auditweights_userOwner_TEXT
KFold(n_splits=5, random_state=430, shuffle=True)
Starting train: 2019-03-05 10:50:46.684719


KeyboardInterrupt: 

In [54]:
test = create_features(test_data)

In [55]:
model = LgbModel({
    #'num_threads': 4,
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 20,
    'lambda_l2': 0.0,
    'num_leaves': 20,
    'learning_rate': 0.15,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 1000,
    #'max_depth': 4,
    'verbose': 0
})
model.fit(train)
pred = model.predict(test)

Starting train: 2019-03-04 23:34:25.850021


In [56]:
submit = predict_to_submit(test, pred)
submit.to_csv(output_path + "/collabSubmit.csv.gz", header=False, compression='gzip')